In [1]:
import numpy as np
import pandas as pd
import json

from bokeh.plotting import figure, output_file, show
from bokeh.layouts import row,column
from bokeh.models import ColumnDataSource, PanTool,ResetTool,BoxZoomTool,WheelZoomTool,UndoTool,HoverTool
from bokeh.palettes import Spectral

from Project import dataCollection
from src.algorithm import EMPTY_BLOCK_MOVEMENT
from src.runner import paralleledRunner

In [2]:
# Not returning anything, but output a file
def plotting(df:pd.DataFrame) -> None:
    # Define tools needed
    tools = [PanTool(),ResetTool(),BoxZoomTool(),WheelZoomTool(dimensions='width'),WheelZoomTool(dimensions='height'),UndoTool()]
    
    #Define colors
    #colors = Category20c[20]+Category20b[20]
    colors = Spectral[11]
    
    # Define datascource
    datasource_dict = {'x': df.C.unique()}
    for num_of_y in range(0,len(df.MN.unique())):
        datasource_dict['y'+str(num_of_y)] = list(df.loc[df.MN == df.MN.unique()[num_of_y],'steps'].apply(lambda x: np.array(x).mean()))
        datasource_dict['y'+str(num_of_y+len(df.MN.unique()))] = list(df.loc[df.MN == df.MN.unique()[num_of_y],'density'])
    datasource = ColumnDataSource(datasource_dict)
    
    # First figure showing the relationship between C and Steps with given M,N combination, Second show C-Density
    p1 = figure(background_fill_color="white", plot_width=1600, plot_height=800, title = "C-Steps Plot",tools=tools)
    p2 = figure(background_fill_color="white", plot_width=1600, plot_height=800, title = "C-Density Plot",tools=tools)
    for num_of_y in range(0,len(df.MN.unique())):
        p1.line('x', 'y'+str(num_of_y), source=datasource, legend_label='(M,N) = '+str(df.MN.unique()[num_of_y]),muted_alpha=0.2,color=colors[num_of_y])
        p2.line('x', 'y'+str(num_of_y+len(df.MN.unique())), source=datasource, legend_label='(M,N) = '+str(df.MN.unique()[num_of_y]),muted_alpha=0.2,color=colors[num_of_y])
        for i in range(0,len(df.C.unique())):
            y1 = list(df.loc[df.MN == df.MN.unique()[num_of_y],'steps'])[i]
            p1.circle(x=df.C.unique()[i], y=y1, color=colors[num_of_y],muted_alpha=0.2)
            
    p1.legend.click_policy="mute"
    p2.legend.click_policy="mute"
    
    #ouput file
    show(column(p1, p2))
    output_file("stats.html", title="stats")

In [3]:
df = pd.read_csv('Output_noindex.csv')
df['steps'] = df.steps.apply(lambda x: json.loads(x))
df = df[(df['B'] == 1)&(df.MN.isin(df.MN.unique()[[0,5,10,15,20,25]])==True)]
df

,MN,C,B,steps,density
0,"(20, 20)",1.0,1.0,"[26, 20, 61, 84, 43, 43, 54, 0, 12, 12, 0, 160...",0.950000
15,"(20, 20)",2.0,1.0,"[44, 101, 0, 81, 75, 17, 38, 57, 58, 0, 3, 30,...",0.900000
30,"(20, 20)",3.0,1.0,"[23, 42, 12, 37, 20, 51, 61, 5, 42, 56, 53, 4,...",0.850000
45,"(20, 20)",4.0,1.0,"[18, 0, 31, 23, 14, 21, 21, 12, 42, 0, 13, 39,...",0.800000
62,"(80, 40)",1.0,1.0,"[289, 702, 261, 332, 418, 243, 392, 308, 216, ...",0.975000
77,"(80, 40)",2.0,1.0,"[59, 207, 124, 173, 268, 395, 93, 23, 284, 221...",0.950000
92,"(80, 40)",3.0,1.0,"[110, 36, 139, 99, 75, 196, 38, 35, 126, 113, ...",0.925000
107,"(80, 40)",4.0,1.0,"[168, 139, 18, 44, 32, 38, 63, 87, 93, 98, 142...",0.900000
181,"(80, 160)",1.0,1.0,"[3789, 1504, 3235, 4798, 1134, 3168, 1319, 396...",0.993750
196,"(80, 160)",2.0,1.0,"[2976, 2837, 1495, 1810, 2178, 1774, 3028, 106...",0.987500


In [4]:
df.to_csv('Output_noindex.csv', index = False)
df.to_csv('Output.csv')

Start : This command cannot be run due to the error: The system cannot find the file specified.
At line:1 char:1
+ Start "file:///tmp/tmp0k10kw7r.html"
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 
